In [2]:
using Pkg;
Pkg.activate(".")
Pkg.add("Plots")
Pkg.add("BenchmarkTools")

using Plots
using BenchmarkTools

  Activating environment at `~/Documents/GitHub/Phys215-202122-1/05-Fast-Data/Project.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/Phys215-202122-1/05-Fast-Data/Project.toml`
  No Changes to `~/Documents/GitHub/Phys215-202122-1/05-Fast-Data/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/Phys215-202122-1/05-Fast-Data/Project.toml`
  No Changes to `~/Documents/GitHub/Phys215-202122-1/05-Fast-Data/Manifest.toml`


In [3]:
ca = rand([0,1],10)

10-element Vector{Int64}:
 0
 0
 1
 1
 0
 0
 1
 1
 1
 1

In [29]:
@inline function rule(r::Vector{Int})
    return Dict(
     [1,1,1]=>r[1]
    ,[1,1,0]=>r[2]
    ,[1,0,1]=>r[3]
    ,[1,0,0]=>r[4]
    ,[0,1,1]=>r[5]
    ,[0,1,0]=>r[6]
    ,[0,0,1]=>r[7]
    ,[0,0,0]=>r[8]
    )
end

@inline function rule(r::Int) #output is rule
    #convert r into rr, to vector of binary digits
    #rr = digits( parse(Int,bitstring(r),base=2), base=2, pad=8 )
    #println("Creating rule $(r)..")
    #println("  R = ",string(r,base=2,pad=8))
    rr = reverse( digits( parse(Int,string(r,base=2,pad=8)), pad=8) )
    return rule(rr)
end

rule (generic function with 2 methods)

In [30]:
wolfram = rand(0:254)
println(wolfram)

R = rule(wolfram)
R = rule(30)

185


Dict{Vector{Int64}, Int64} with 8 entries:
  [1, 1, 1] => 0
  [0, 0, 0] => 0
  [1, 1, 0] => 0
  [0, 0, 1] => 1
  [1, 0, 0] => 1
  [0, 1, 1] => 1
  [1, 0, 1] => 0
  [0, 1, 0] => 1

In [6]:
R[[1,1,0]]

0

In [89]:
function apply!(vout::Vector{Int}, vin::Vector{Int}, rule::Dict{Vector{Int}, Int})
    L = length(vin);
    vout[1] = R[ vin[[L,1,2]] ]
    vout[L] = R[ vin[[L-1,L,1]] ]
    for i in 2:L-1 #ends not included
        vout[i] = R[vin[i-1:i+1]]
    end
    return nothing;
end

@views function apply_sub!(vout::SubArray, vin::SubArray, rule::Dict{Vector{Int}, Int})
    L = length(vin);
    vout[1] = R[ vin[[L,1,2]] ]
    vout[L] = R[ vin[[L-1,L,1]] ]
    for i in 2:L-1 #ends not included
        vout[i] = R[vin[i-1:i+1]]
    end
    return nothing;
end

apply_sub! (generic function with 2 methods)

In [33]:
function apply(vin::Vector{Int}, rule::Dict{Vector{Int}, Int})
    vout = copy(vin);
    L = length(vin);
    vout[1] = R[ vin[[L,1,2]] ]
    vout[L] = R[ vin[[L-1,L,1]] ]
    for i in 2:L-1 #ends not included
        vout[i] = R[ vin[i-1:i+1] ]
    end
    return vout;
end

function apply(vin::Vector{Int}, rule::Dict{Vector{Int}, Int}, p::Float64)
    vout = copy(vin);
    L = length(vin);
    vout[1] = R[ vin[[L,1,2]] ]
    vout[L] = R[ vin[[L-1,L,1]] ]
    for i in 2:L-1 #ends not included
        vout[i] = R[ vin[i-1:i+1] ]
        p>0.0 && vout[i]==1 && ( vout[i] = rand()<1-p ? 1 : 0 );
    end
    return vout;
end

apply (generic function with 2 methods)

In [43]:
function tester0()
    L = 256
    T = 10L
    M = zeros(Int,L,T);
    
    # Initialize..
    M[Int(L/2),1] = 1;
    M[:,1] = rand([0,1],L);
    # ..ensure boundaries are zero..
    M[1,1] = 0;
    M[L,1] = 0;
    
    R = rule(30)
    
    # Apply rule R..
    for t in 1:T-1
        #M[:,t+1] = apply(M[:,t], R, 0.39) #0.38)
        #M[:,t+1] = apply(M[:,t], R, 0.0001)
        M[:,t+1] = apply(M[:,t], R)
    #    println(M[:,t+1])
    end
        
    return nothing;
end

tester0 (generic function with 1 method)

In [12]:
tester0()

In [35]:
function apply_simd(vin::Vector{Int}, rule::Dict{Vector{Int}, Int})
    vout = copy(vin);
    L = length(vin);
    vout[1] = R[ vin[[L,1,2]] ]
    vout[L] = R[ vin[[L-1,L,1]] ]
    @simd for i in 2:L-1 #ends not included
        vout[i] = R[ vin[i-1:i+1] ]
    end
    return vout;
end


function tester0a()
    L = 256
    T = 10L
    M = zeros(Int,L,T);
    
    # Initialize..
    M[Int(L/2),1] = 1;
    M[:,1] = rand([0,1],L);
    # ..ensure boundaries are zero..
    M[1,1] = 0;
    M[L,1] = 0;
    
    R = rule(30)
    
    # Apply rule R..
    for t in 1:T-1
        M[:,t+1] = apply_simd(M[:,t], R)
    end
    
    return nothing;
end

tester0a (generic function with 1 method)

In [52]:
L = 8
T = 10L
M = zeros(Int,L,T);

# Initialize..
M[Int(L/2),1] = 1;
M[:,1] = rand([0,1],L);
# ..ensure boundaries are zero..
M[1,1] = 0;
M[L,1] = 0;

#println(M[:,1])
#println(M[:,2])

R = rule(30)
println(R)

Dict([1, 1, 1] => 0, [0, 0, 0] => 0, [1, 1, 0] => 0, [0, 0, 1] => 1, [1, 0, 0] => 1, [0, 1, 1] => 1, [1, 0, 1] => 0, [0, 1, 0] => 1)


In [55]:
println(M[:,1])
v = M[:,2]
apply!(v,M[:,1],R)
M[:,2] = v
println(M[:,2])
println("---")
println(v)

[0, 1, 0, 0, 1, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 0, 0]
---
[1, 1, 1, 1, 1, 1, 0, 0]


In [464]:
M[:,2]

8-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0

In [108]:
function test!(vout::Vector{Int}, vin::Vector{Int})
    vout[3] = vin[3]+1;
end

test! (generic function with 1 method)

In [109]:
v = zeros(Int,5)
test!(v,v)
v

5-element Vector{Int64}:
 0
 0
 1
 0
 0

# Application `!` ensuring allocation reuse

In [97]:
@views function tester1()
    L = 256
    T = 10L
    M = zeros(Int,L,T);
    
    # Initialize..
    M[Int(L/2),1] = 1;
    M[:,1] = rand([0,1],L);
    # ..ensure boundaries are zero..
    M[1,1] = 0;
    M[L,1] = 0;
    
    R = rule(30)
    
    # Apply rule R..
    # .. allocate..
    for t in 1:T-1
        apply_sub!(M[:,t+1], M[:,t],R)
    end
        
    return nothing;
end

tester1 (generic function with 1 method)

In [90]:
function tester2()
    L = 256
    T = 10L
    M = zeros(Int,L,T);
    
    # Initialize..
    M[Int(L/2),1] = 1;
    M[:,1] = rand([0,1],L);
    # ..ensure boundaries are zero..
    M[1,1] = 0;
    M[L,1] = 0;
    
    R = rule(30)
    
    # Apply rule R..
    # .. allocate..
    @views for t in 1:T-1
        vou = M[:,t+1]
        vin = M[:,t]
        apply_sub!(vou, vin, R)
    end
    
    return nothing;
end

tester2 (generic function with 1 method)

In [59]:
tester1()

In [102]:
mark0 = @benchmark tester0()

BenchmarkTools.Trial: 40 samples with 1 evaluation.
 Range (min … max):  118.753 ms … 143.865 ms  ┊ GC (min … max): 1.21% … 1.92%
 Time  (median):     125.504 ms               ┊ GC (median):    2.30%
 Time  (mean ± σ):   128.228 ms ±   6.560 ms  ┊ GC (mean ± σ):  2.13% ± 0.45%

        ▁  ▁ █ █▁▁ ▁    ▁▁                                       
  ▆▁▁▁▁▁█▆▁█▆█▆███▁█▁▆▁▁██▆▁▁▁▁▆▆▁▁▁▆▁▆▁▆▆▁▁▆▁▁▁▁▁▆▁▁▁▆▁▁▁▁▆▆▁▆ ▁
  119 ms           Histogram: frequency by time          144 ms <

 Memory estimate: 86.14 MiB, allocs estimate: 665360.

In [98]:
mark1 = @benchmark tester1()

BenchmarkTools.Trial: 47 samples with 1 evaluation.
 Range (min … max):  102.100 ms … 112.958 ms  ┊ GC (min … max): 1.92% … 3.48%
 Time  (median):     109.285 ms               ┊ GC (median):    1.81%
 Time  (mean ± σ):   108.458 ms ±   2.806 ms  ┊ GC (mean ± σ):  2.19% ± 0.73%

                                                 █               
  ▄▁▇▁▁▁▄▁▄▁▁▁▄▁▁▄▁▄▄▁▁▄▁▄▁▄▄▇▁▁▄▁▁▁▄▁▄▄▇▇▄▁▇▇▄▇▇█▄▇▄▄▁▁▁▄▄▁▁▁▄ ▁
  102 ms           Histogram: frequency by time          113 ms <

 Memory estimate: 55.45 MiB, allocs estimate: 1315091.

In [103]:
median(mark1.times)/median(mark0.times)

0.8707740527896833

In [40]:
mark0a = @benchmark tester0a()

BenchmarkTools.Trial: 39 samples with 1 evaluation.
 Range (min … max):  118.098 ms … 137.120 ms  ┊ GC (min … max): 1.08% … 1.99%
 Time  (median):     129.536 ms               ┊ GC (median):    2.14%
 Time  (mean ± σ):   129.268 ms ±   5.417 ms  ┊ GC (mean ± σ):  2.03% ± 0.39%

                        ▁             ▁           ▁   ▁       █  
  ▆▁▁▁▁▆▁▁▁▆▆▆▁▆▁▆▁▁▁▆▁▁█▆▁▆▆▆▁▆▁▆▁▆▁▆█▆▁▆▆▆▆▁▁▆▆▁█▁▆▁█▁▁▆▆▁▁▆█ ▁
  118 ms           Histogram: frequency by time          137 ms <

 Memory estimate: 86.14 MiB, allocs estimate: 665360.

In [41]:
median(mark0a.times)/median(mark0.times)

1.164886390596164

In [100]:
mark2 = @benchmark tester2()

BenchmarkTools.Trial: 44 samples with 1 evaluation.
 Range (min … max):  106.473 ms … 137.755 ms  ┊ GC (min … max): 1.79% … 1.71%
 Time  (median):     113.058 ms               ┊ GC (median):    1.88%
 Time  (mean ± σ):   114.221 ms ±   6.557 ms  ┊ GC (mean ± σ):  2.18% ± 0.71%

  ▄▁   ▄▄▁▁   █▄  ▄▁▁                                            
  ██▆▁▆████▆▆▁██▁▁███▆▁▆▁▆▆▆▁▆▆▆▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▆ ▁
  106 ms           Histogram: frequency by time          138 ms <

 Memory estimate: 55.45 MiB, allocs estimate: 1315091.

In [104]:
median(mark2.times)/median(mark0.times)

0.9008350365563247

In [63]:
m = rand(5,5)

5×5 Matrix{Float64}:
 0.469011  0.715243  0.445719  0.0491054  0.0885741
 0.529462  0.766705  0.906707  0.358262   0.654144
 0.583304  0.252865  0.807627  0.164025   0.523424
 0.231291  0.18598   0.713221  0.968958   0.186563
 0.557086  0.56739   0.556388  0.885632   0.319981

In [64]:
v = m[:,1]

5-element Vector{Float64}:
 0.4690114950934292
 0.5294618071184556
 0.5833042780564572
 0.2312910362133458
 0.5570858346828655

In [65]:
v[1] = 1.0

1.0

In [67]:
v

5-element Vector{Float64}:
 1.0
 0.5294618071184556
 0.5833042780564572
 0.2312910362133458
 0.5570858346828655

# `Particle` trajectory

Here's a simple computation of a trajectory of a particle of unit mass under the uniform gravity.

Use the usual $xy$-axes with $\vec{r}=\langle{x,y}\rangle$ and that $y$ is positive "upwards" giving $\vec{g} = \langle{0,-g}\rangle$.

In [38]:
struct Particle
    r::Vector
    v::Vector
    a::Vector
end

g_CONST = [0.0,-9.80]

Particle() = Particle([0.0,10.0], [0.0,10.0], g_CONST) #default constructor method

Particle

In [39]:
p = Particle()

Particle([0.0, 10.0], [0.0, 10.0], [0.0, -9.8])

In [40]:
p.r = [0.0, 0.0]

LoadError: setfield! immutable struct of type Particle cannot be changed

In [41]:
function move_im(pp::Particle, dt=0.01)
    r = zeros(2);
    v = zeros(2);
    r .= pp.r .+ pp.v .* dt + (0.5 .* pp.a .* (dt * dt))
    v .= pp.v .+ pp.a .* dt
    return Particle(r,v,pp.a)
end

move_im (generic function with 2 methods)

In [42]:
function move_ex(pp::Particle, dt=0.01)
    r = zeros(2);
    v = zeros(2);
    r[1] = pp.r[1] + pp.v[1] * dt + (0.5 * pp.a[1] * (dt * dt))
    r[2] = pp.r[2] + pp.v[2] * dt + (0.5 * pp.a[2] * (dt * dt))
    v[1] = pp.v[1] + pp.a[1] * dt
    v[2] = pp.v[2] + pp.a[2] * dt
    return Particle(r,v,pp.a)
end

move_ex (generic function with 2 methods)

In [53]:
mark0im = @benchmark begin
    p = Particle()
    pnew = Particle()
    for _ in 1:100
        p = move_im(p);
#        p = pnew;
    end
end

mark0ex = @benchmark begin
    p = Particle()
    pnew = Particle()
    for _ in 1:100
        p = move_ex(p);
#        p = pnew;
    end
end

println( "Advantage of ex- / implicit : ", round( median(mark0im.times)/median(mark0ex.times), digits=3 ) )

Advantage of ex- / implicit : 3.994


In [44]:
mutable struct MParticle
    r::Vector
    v::Vector
    a::Vector
end

g_CONST = [0.0,-9.80]

MParticle() = MParticle([0.0,10.0], [0.0,10.0], g_CONST)

MParticle

In [45]:
mp = MParticle()

MParticle([0.0, 10.0], [0.0, 10.0], [0.0, -9.8])

In [46]:
display(mp)
mp.r = [0.0, 210.0]
display(mp)

MParticle([0.0, 10.0], [0.0, 10.0], [0.0, -9.8])

MParticle([0.0, 210.0], [0.0, 10.0], [0.0, -9.8])

In [47]:
function move_im!(pp::MParticle, dt=0.01)
    pp.r .= pp.r .+ (pp.v .* dt) .+ (0.5 .* pp.a .* (dt * dt))
    pp.v .= pp.v .+ (pp.a[2] .* dt)
    return nothing;
end

move_im! (generic function with 2 methods)

In [48]:
function move_ex!(pp::MParticle, dt=0.01)
    pp.r[1] = pp.r[1] + (pp.v[1] * dt) + (0.5 * pp.a[1] * (dt * dt))
    pp.r[2] = pp.r[2] + (pp.v[2] * dt) + (0.5 * pp.a[2] * (dt * dt))
    pp.v[1] = pp.v[1] + (pp.a[1] * dt)
    pp.v[2] = pp.v[2] + (pp.a[2] * dt)
    return nothing;
end

move_ex! (generic function with 2 methods)

In [54]:
mark1ex = @benchmark begin
    p = MParticle()
    for _ in 1:100
        move_ex!(p);
    end
end

mark1im = @benchmark begin
    p = MParticle()
    for _ in 1:100
        move_im!(p);
    end
end

println( "Immutable: Advantage of ex- / implicit : ", round( median(mark0im.times)/median(mark0ex.times), digits=3 ) )
println( "Mutable  : Advantage of ex- / implicit : ", round( median(mark1im.times)/median(mark1ex.times), digits=3 ) )
println( "Explicit : Advantage of Mutable / Immutable : ", round( median(mark0ex.times)/median(mark1ex.times), digits=3 ) )
println( "Implicit : Advantage of Mutable / Immutable : ", round( median(mark0im.times)/median(mark1im.times), digits=3 ) )

Immutable: Advantage of ex- / implicit : 3.994
Mutable  : Advantage of ex- / implicit : 5.211
Explicit : Advantage of Mutable / Immutable : 1.196
Implicit : Advantage of Mutable / Immutable : 0.917


# Pass by object ("by sharing")

+ Objects are passed by copying to local variables the address or reference that object variables points to
+ Immutable variables as objects therefore are passed by value since they contain values, rather than reference

In [6]:
name = "My name"

function change_name(n::String)
    n *= " is John."
    println("change_name(): name = ", n)
    return nothing
end

mutable struct Name
    val::String
end

change_name(name)
println("now, name = ", name)

name = Name("My name")
function change_name!(n::Name)
    n.val *= " is Johnny."
    println("change_name(): name = ", n.val)
    return nothing
end

change_name!(name)
println("now, name = ", name.val)

change_name(): name = My name is John.
now, name = My name
change_name(): name = My name is Johnny.
now, name = My name is Johnny.


In [17]:
struct CoolNumbers
    num::Vector{Float64}
end

x = CoolNumbers( rand(3) )
y = CoolNumbers( rand(4) )

println("x = ", x)
println("y = ", y)

x.num[3] = 3.0

println("x = ", x)
println("y = ", y)

x = CoolNumbers([0.016303370673339757, 0.1172113440415814, 0.6162099207980105])
y = CoolNumbers([0.7076542822068044, 0.4047643644559309, 0.5571097005435184, 0.3645053956512674])
x = CoolNumbers([0.016303370673339757, 0.1172113440415814, 3.0])
y = CoolNumbers([0.7076542822068044, 0.4047643644559309, 0.5571097005435184, 0.3645053956512674])


In [18]:
function changeNos!(x::CoolNumbers)
    x.num[:] .= rand(3)
    x.num[3] = 3.0
    return nothing
end

println(x)
changeNos!(x)
println(x)

CoolNumbers([0.016303370673339757, 0.1172113440415814, 3.0])
CoolNumbers([0.14082142257177832, 0.16692381066472706, 3.0])


**Think:** Julia objects can be thought of Namespaces **of references** (not of values)